In [0]:
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/audio_test.h5"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/audio_train.h5"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/text_test_emb.h5"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/text_train_emb.h5"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/video_test.h5"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/video_train.h5"

!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Test_labels_angry.csv"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Test_labels_disgust.csv"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Test_labels_fear.csv"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Test_labels_happy.csv"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Test_labels_sad.csv"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Test_labels_surprise.csv"

!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Train_labels_angry.csv"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Train_labels_disgust.csv"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Train_labels_fear.csv"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Train_labels_happy.csv"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Train_labels_sad.csv"
!wget "http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/labels/mosi2uni_Train_labels_surprise.csv"

In [0]:
import numpy as np
import h5py
import pandas as pd

Test_labels_surprise = pd.read_csv("mosi2uni_Test_labels_surprise.csv",header=None)
Test_labels_angry = pd.read_csv("mosi2uni_Test_labels_angry.csv",header=None)
Test_labels_disgust = pd.read_csv("mosi2uni_Test_labels_disgust.csv",header=None)
Test_labels_fear = pd.read_csv("mosi2uni_Test_labels_fear.csv",header=None)
Test_labels_happy = pd.read_csv("mosi2uni_Test_labels_happy.csv",header=None)
Test_labels_sad = pd.read_csv("mosi2uni_Test_labels_sad.csv",header=None)

Train_labels_surprise = pd.read_csv("mosi2uni_Train_labels_surprise.csv",header=None)
Train_labels_angry = pd.read_csv("mosi2uni_Train_labels_angry.csv",header=None)
Train_labels_disgust = pd.read_csv("mosi2uni_Train_labels_disgust.csv",header=None)
Train_labels_fear = pd.read_csv("mosi2uni_Train_labels_fear.csv",header=None)
Train_labels_happy = pd.read_csv("mosi2uni_Train_labels_happy.csv",header=None)
Train_labels_sad = pd.read_csv("mosi2uni_Train_labels_sad.csv",header=None)

In [0]:
video_train = h5py.File("video_train.h5","r")
print(list(video_train.keys()))
video_train = np.array(video_train.get('d1'))
video_train.shape

['d1']


(15290, 20, 35)

In [0]:
video_test = h5py.File("video_test.h5","r")
print(list(video_test.keys()))
video_test = np.array(video_test.get('d1'))
video_test.shape

['d1']


(4832, 20, 35)

In [0]:
audio_train = h5py.File("audio_train.h5","r")
print(list(audio_train.keys()))
audio_train = np.array(audio_train.get('d1'))
audio_train.shape

['d1']


(15290, 20, 74)

In [0]:
audio_test = h5py.File("audio_test.h5","r")
print(list(audio_test.keys()))
audio_test = np.array(audio_test.get('d1'))
audio_test.shape

['d1']


(4832, 20, 74)

In [0]:
text_train_emb = h5py.File("text_train_emb.h5","r")
print(list(text_train_emb.keys()))
text_train_emb = np.array(text_train_emb.get('d1'))
text_train_emb.shape

['d1']


(15290, 20, 300)

In [0]:
text_test_emb = h5py.File("text_test_emb.h5","r")
print(list(text_test_emb.keys()))
text_test_emb = np.array(text_test_emb.get('d1'))
text_test_emb.shape

['d1']


(4832, 20, 300)

This is example of Early Feature-wise Fusion where I am concatenaing features for three modalities and than train LSTM+DNN model on concatenated features.

In [0]:
concat_train = np.concatenate((video_train,audio_train),axis=-1)
concat_train = np.concatenate((concat_train, text_train_emb),axis=-1)
concat_train.shape

(15290, 20, 409)

In [0]:
concat_test = np.concatenate((video_test,audio_test),axis=-1)
concat_test = np.concatenate((concat_test, text_test_emb),axis=-1)
concat_test.shape

(4832, 20, 409)

In [0]:
import keras
from google.colab import files	
from keras.models import load_model
from keras.models import Model,Sequential,Model
from keras.layers import LSTM,Dense,Input, Flatten,Dropout
from keras import callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import plot_model

In [0]:
i = Input(shape=(20,409))
x = LSTM(units=256,return_sequences=True)(i)
x = LSTM(units=256,return_sequences=True)(x)
x = LSTM(units=128,return_sequences=True)(x)
x = LSTM(units=128,return_sequences=True)(x)
x = LSTM(units=64,return_sequences=True)(x)
x = LSTM(units=64,return_sequences=True)(x)
x = LSTM(units=32,return_sequences=True)(x)
x = LSTM(units=32,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(units=64,activation='relu')(x)
x = Dropout(rate=0.15)(x)
x = Dense(units=64,activation='relu')(x)
o = Dense(1,activation='sigmoid')(x)

In [0]:
model_rnn_happy = Model(inputs=i,outputs=o)
model_rnn_angry = Model(inputs=i,outputs=o)
model_rnn_fear = Model(inputs=i,outputs=o)
model_rnn_surprise = Model(inputs=i,outputs=o)
model_rnn_disgust = Model(inputs=i,outputs=o)
model_rnn_sad = Model(inputs=i,outputs=o)

In [0]:
model_rnn_happy.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 20, 409)           0         
_________________________________________________________________
lstm_49 (LSTM)               (None, 20, 256)           681984    
_________________________________________________________________
lstm_50 (LSTM)               (None, 20, 256)           525312    
_________________________________________________________________
lstm_51 (LSTM)               (None, 20, 128)           197120    
_________________________________________________________________
lstm_52 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
lstm_53 (LSTM)               (None, 20, 64)            49408     
_________________________________________________________________
lstm_54 (LSTM)               (None, 20, 64)            33024     
__________

In [0]:
plot_model(model_rnn_happy, to_file='early_fusion_happy.png',show_layer_names=True, show_shapes=True)
files.download('early_fusion_happy.png')

In [39]:
model_rnn_happy.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("early_fusion_happy.h5",monitor='val_loss',verbose=1)
history_rnn_happy = model_rnn_happy.fit(concat_train,Train_labels_happy,batch_size=64,epochs=20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
print(model_rnn_happy.evaluate(concat_test,Test_labels_happy))
print("Saving the model=========================================================")
saved_rnn_happy = load_model('early_fusion_happy.h5')
print("Evaluating saved model===================================================")
saved_rnn_happy.evaluate(concat_test,Test_labels_happy)

W0624 08:03:22.033298 139711233992576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0624 08:03:22.069317 139711233992576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0624 08:03:22.077915 139711233992576 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 83s 6ms/step - loss: 0.6604 - acc: 0.6067 - val_loss: 0.6849 - val_acc: 0.5899

Epoch 00001: saving model to early_fusion_happy.h5
Epoch 2/20
13761/13761 [==============================] - 71s 5ms/step - loss: 0.6402 - acc: 0.6426 - val_loss: 0.6596 - val_acc: 0.6102

Epoch 00002: saving model to early_fusion_happy.h5
Epoch 3/20
13761/13761 [==============================] - 71s 5ms/step - loss: 0.6237 - acc: 0.6601 - val_loss: 0.6512 - val_acc: 0.6246

Epoch 00003: saving model to early_fusion_happy.h5
Epoch 4/20
13761/13761 [==============================] - 72s 5ms/step - loss: 0.6144 - acc: 0.6686 - val_loss: 0.6547 - val_acc: 0.6207

Epoch 00004: saving model to early_fusion_happy.h5
Epoch 5/20
13761/13761 [==============================] - 71s 5ms/step - loss: 0.6034 - acc: 0.6779 - val_loss: 0.6765 - val_acc: 0.6174

Epoch 00005: saving model to early_fusion_happy.h5
Epoch 

[0.6538063246288047, 0.668046357615894]

In [0]:
files.download('early_fusion_happy.h5')

In [41]:
model_rnn_sad.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("early_fusion_sad.h5",monitor='val_loss',verbose=1)
history_rnn_sad = model_rnn_sad.fit(concat_train,Train_labels_sad,batch_size=64,epochs=20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
print(model_rnn_sad.evaluate(concat_test,Test_labels_sad))
print("Saving the model=========================================================")
saved_rnn_sad = load_model('early_fusion_sad.h5')
print("Evaluating saved model===================================================")
saved_rnn_sad.evaluate(concat_test,Test_labels_sad)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 82s 6ms/step - loss: 0.5718 - acc: 0.7362 - val_loss: 0.5297 - val_acc: 0.7626

Epoch 00001: saving model to early_fusion_sad.h5
Epoch 2/20
13761/13761 [==============================] - 73s 5ms/step - loss: 0.5502 - acc: 0.7425 - val_loss: 0.5245 - val_acc: 0.7626

Epoch 00002: saving model to early_fusion_sad.h5
Epoch 3/20
13761/13761 [==============================] - 73s 5ms/step - loss: 0.5463 - acc: 0.7418 - val_loss: 0.5267 - val_acc: 0.7626

Epoch 00003: saving model to early_fusion_sad.h5
Epoch 4/20
13761/13761 [==============================] - 72s 5ms/step - loss: 0.5289 - acc: 0.7438 - val_loss: 0.5441 - val_acc: 0.7475

Epoch 00004: saving model to early_fusion_sad.h5
Epoch 5/20
13761/13761 [==============================] - 72s 5ms/step - loss: 0.5076 - acc: 0.7540 - val_loss: 0.5528 - val_acc: 0.7619

Epoch 00005: saving model to early_fusion_sad.h5
Epoch 6/20
13761

[0.6280752976011756, 0.6988824503311258]

In [0]:
files.download('early_fusion_sad.h5')

In [44]:
model_rnn_surprise.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("early_fusion_surprise.h5",monitor='val_loss',verbose=1)
history_rnn_surprise = model_rnn_surprise.fit(concat_train,Train_labels_surprise,batch_size=64,epochs=20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
print(model_rnn_surprise.evaluate(concat_test,Test_labels_surprise))
print("Saving the model=========================================================")
saved_rnn_surprise = load_model('early_fusion_surprise.h5')
print("Evaluating saved model===================================================")
saved_rnn_surprise.evaluate(concat_test,Test_labels_surprise)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 83s 6ms/step - loss: 0.3353 - acc: 0.8948 - val_loss: 0.2978 - val_acc: 0.9091

Epoch 00001: saving model to early_fusion_surprise.h5
Epoch 2/20
13761/13761 [==============================] - 72s 5ms/step - loss: 0.3254 - acc: 0.8966 - val_loss: 0.2991 - val_acc: 0.9091

Epoch 00002: saving model to early_fusion_surprise.h5
Epoch 3/20
13761/13761 [==============================] - 72s 5ms/step - loss: 0.3166 - acc: 0.8971 - val_loss: 0.3197 - val_acc: 0.9091

Epoch 00003: saving model to early_fusion_surprise.h5
Epoch 4/20
13761/13761 [==============================] - 72s 5ms/step - loss: 0.2996 - acc: 0.9007 - val_loss: 0.3134 - val_acc: 0.9071

Epoch 00004: saving model to early_fusion_surprise.h5
Epoch 5/20
13761/13761 [==============================] - 72s 5ms/step - loss: 0.2744 - acc: 0.9070 - val_loss: 0.4410 - val_acc: 0.8973

Epoch 00005: saving model to early_fusion_sur

[0.38686924131698164, 0.8952814569536424]

In [0]:
files.download('early_fusion_surprise.h5')